In [23]:
import kagglehub
import os
import zipfile
import pandas as pd

In [10]:
# checking the current working directory
%pwd

'a:\\NLP Projects\\Chatbot\\python_code\\Experiments'

In [8]:
## Moving a folder back
cd ..

SyntaxError: invalid syntax (1397965987.py, line 2)

In [21]:
dataset_path = "xavierberge/florist-shop-dataset"
output_dir = "a:/NLP Projects/Chatbot/python_code/Dataset"

In [14]:
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Download the dataset
print(f"Downloading dataset: {dataset_path}")
try:
    zip_path = kagglehub.dataset_download(dataset_path, output_dir)
    print(f"Dataset downloaded to: {zip_path}")

    # Unzip the dataset
    print(f"Unzipping dataset to: {output_dir}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

    print("Download and extraction complete!")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Unzip the dataset
print(f"Unzipping dataset to: {output_dir}")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

In [35]:
# Change the working directory
new_directory = "a:/NLP Projects/Chatbot/python_code/Dataset"  # Replace with your desired path
os.chdir(new_directory)

In [37]:
# Print the current working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: a:\NLP Projects\Chatbot\python_code\Dataset


In [ ]:
python_code\Dataset

In [47]:
dataset1 = pd.read_csv('Accounts.csv')
hirerachy = pd.read_csv('Plant_Hirecrachy.csv',encoding='Latin-1')
sales = pd.read_csv('plant_sales.csv')

In [40]:
dataset.head()

,country_code,Account,Master_id,Account_id,latitude2,longitude,country2,Postal_code,street_name,Street_number
0,CA,"Gerlach, Cassin and Parker",1599,1599-E6G-78670,45.989139,-67.241222,Canada,E6G,Oneill,78670
1,CN,Ziemann LLC,1227,1227--40,37.943121,115.217658,China,0,Kipling,40
2,CN,"Medhurst, Aufderhar and Mosciski",1857,1857--758,26.170171,118.190467,China,0,Glendale,758
3,AR,Larson-Fritsch,1793,1793-6140-77,-33.916294,-64.389795,Argentina,6140,Manley,77
4,MX,Jast-Quigley,1725,1725-40230-69,18.559181,-99.598300,Mexico,40230,Leroy,69


In [45]:
hirerachy.head()

,Product_Family,Product_Family_Id,Product_Group,Product_Group_id,Product_Name,Product_Name_id,Product_Size,Produt_Type
0,Cucurbitaceae,1051,Gray's Bur Cucumber,3293,Chamaesyce celastroides (Boiss.) Croizat & O. ...,2000,Small,Landscape
1,Platanaceae,1138,Arizona Sycamore,3658,Tetraplasandra waimeae Wawra,2001,Medium,Outdoor
2,Scrophulariaceae,1167,Little Elephantshead,3876,Iris ?ónelsonii Randolph,2002,Small,Outdoor
3,Euphorbiaceae,1065,'ekoko,3357,Acanthus L.,2003,Large,Outdoor
4,Amaranthaceae,1006,Philoxerus,3017,Aplectrum hyemale (Muhl. ex Willd.) Torr.,2004,Medium,Landscape


In [48]:
sales.head()

,Product_id,Sales_USD,quantity,Price_USD,COGS_USD,Date_Time,Account_id
0,2625,16156.56,502.00,32.184382,14104.67688,3/29/2024,1599-E6G-78670
1,2569,13831.29,679.92,20.342526,10470.28653,7/13/2022,1227--40
2,2195,17402.14,925.66,18.799710,10528.29470,4/5/2023,1857--758
3,2530,15866.42,809.78,19.593495,12883.53304,9/24/2022,1793-6140-77
4,2889,6200.48,311.43,19.909707,5388.21712,10/19/2023,1725-40230-69
